In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Install the necessary libraries
!pip install transformers
!pip install datasets
!pip install bert_score

     |████████████████████████████████| 3.8 MB 4.2 MB/s 
     |████████████████████████████████| 596 kB 68.6 MB/s 
     |████████████████████████████████| 895 kB 54.7 MB/s 
     |████████████████████████████████| 67 kB 6.6 MB/s 
     |████████████████████████████████| 6.5 MB 63.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 325 kB 4.1 MB/s 
     |████████████████████████████████| 1.1 MB 67.9 MB/s 
     |████████████████████████████████| 212 kB 78.5 MB/s 
     |████████████████████████████████| 136 kB 75.2 MB/s 
     |████████████████████████████████| 127 kB 69.4 MB/s 
     |████████████████████████████████| 271 kB 82.4 MB/s 
     |████████████████████████████████| 94 kB 4.0 MB/s 
     |████████████████████████████████| 144 kB 87.8 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib

In [ ]:

# Import the necessary libraries
import glob
import logging
import re

import os
import pickle
import random
import re
from statistics import mean, median, stdev
import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt
import math
from tqdm import tqdm, trange

from sklearn.model_selection import train_test_split
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import TensorDataset,DataLoader, RandomSampler, SequentialSampler
from torch.utils.data.distributed import DistributedSampler
from tqdm.notebook import tqdm, trange
from pathlib import Path
from transformers import AdamW,AutoModelForCausalLM,AutoTokenizer

from datasets import load_metric # import the automatic evaluation metrics from the dataset libarary 
from nltk.translate.bleu_score import sentence_bleu,corpus_bleu #Import the Blue score metrics

In [ ]:
device =torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu') # Define the computation running ressources

In [ ]:
#import the TG-Redial dataset
import pickle
with open('./drive/MyDrive/Data/English_TGD2_TGD_All_data.pkl','rb') as file:
    TGD_All_data=pickle.load(file)


# Data Pre-processing

In [ ]:
#Remove the movies ids from the topic fields
def remove_movie_from_path_goal(TGD_All_data,typee='train'):
  for i, dial in enumerate(TGD_All_data[typee]):
    goal_path=dial['goal_path']
    for k, v in goal_path.items():
      v=[x for x in v if x[0]!='@']
      goal_path[k]=v
    
    TGD_All_data[typee][i]['goal_path']=goal_path
  return TGD_All_data[typee]

TGD_All_data['train']=remove_movie_from_path_goal(TGD_All_data,typee='train')
TGD_All_data['test']=remove_movie_from_path_goal(TGD_All_data,typee='test')
TGD_All_data['valid']=remove_movie_from_path_goal(TGD_All_data,typee='valid')

In [ ]:
#Combine both topic and intent fields in the same field to concider them as topic information
def create_entent_topic(TGD_All_data=TGD_All_data,typee="train"):

  topic_list=TGD_All_data["real_topics"].values()
  for i,dial in enumerate(TGD_All_data[typee]):
    goal_path=dial["goal_path"]
    new_goal_path={}
    
    for k,v in goal_path.items():
      topics=[x for x in v if x in topic_list]
      intents=[x for x in v if x not in topic_list]
      if(len(topics)==0):
        topics=["no-topic"]
      new_goal_path[k]={"intents":intents,"topics":intents+topics}
    TGD_All_data[typee][i]["goal_path"]=new_goal_path

  return TGD_All_data
TGD_All_data=create_entent_topic(TGD_All_data=TGD_All_data,typee="train")
TGD_All_data=create_entent_topic(TGD_All_data=TGD_All_data,typee="test")
TGD_All_data=create_entent_topic(TGD_All_data=TGD_All_data,typee="valid")





In [ ]:
#Reshape our data to the format that we want by adding different generation controls
def create_path_included_data(data,typee='valid'):
  data_dialogue_rows=[]
  mentioned_movies=[]
  for i,dial in enumerate(data[typee]):
    goal_path=dial['goal_path']
    messages=dial['messages']
    mentionMovies=dial["mentionMovies"]
    movies_interaction=dial["user_history_movies_interaction_ids"]


    visited_profiles=[]
    for j in range(1):
      profile_list=dial["user_profile"]
      personality_list=dial["BIG5"]
      shuffled_profile=random.sample(profile_list,len(profile_list))
      while shuffled_profile in visited_profiles:
        shuffled_profile=random.sample(profile_list,len(profile_list))
      personality= " ".join(str(int(p)) for p in personality_list )
      
      profile=" .".join(p for p in shuffled_profile )
      lines=[]
      lines.append('<big5> '+personality+" </big5>")

      
      lines.append('<profile> '+profile+" </profile>")
      movies_to_add=[]
      '''if len(movies_interaction)==0:
        movies_to_add=[" "]*3

      if len(movies_interaction)==1:
        movies_to_add=[movies_interaction[-1]]*3

      if len(movies_interaction)==2:
        movies_to_add=list(movies_interaction)+[movies_interaction[-1]]

      if len(movies_interaction)>2:
        movies_to_add=list(movies_interaction[-3:])'''

      for i,v in enumerate(messages):
        index=i+1
        if index in mentionMovies:
          curenntmovi=mentionMovies[index]["id"]
          #print(curenntmovi)
          movies_to_add.append(curenntmovi)
          #print(movies_to_add)
          last_three=movies_to_add[-3:]
          random.shuffle(last_three)
          concatenated_movies= " ".join(x for x in last_three)
          #lines[-1]=lines[-1]+" <movies> "+curenntmovi+" </movies>"
          m='<'+v['role'].lower()+'>'+' '+v['content']


        else:
          m='<'+v['role'].lower()+'>'+' '+v['content']
        m=m+' <topic> '+' '.join(x for x in goal_path[v['local_id']]["topics"] )+" </topic> "
        m=re.sub(' +', ' ', m).strip()
        if("@" in m ):
          lin= np.array([word for word in m.split(" ") ])
          indexes=[i for i,v in enumerate(lin) if v[0]=="@"]
          elements=[x for x in list(lin[indexes]) if x is not None]
          mentioned_movies=mentioned_movies+elements

        lines.append(m)
    
    
    
    

    
      data_dialogue_rows.append(lines)
    
  return data_dialogue_rows,mentioned_movies
    

train_rows,train_mentioned_movies=create_path_included_data(TGD_All_data,typee='train')   
test_rows,test_mentioned_movies=create_path_included_data(TGD_All_data,typee='test')   
valid_rows,valid_mentioned_movies=create_path_included_data(TGD_All_data,typee='valid')

In [ ]:
#Shuffel the data samples 
train_rows=random.sample(train_rows,len(train_rows))
test_rows=random.sample(test_rows,len(test_rows))
valid_rows=random.sample(valid_rows,len(valid_rows))


In [ ]:
#Define the special tokens that we are gonna add to the pre-trqined model vocabulary
special_tokens=['<entent>','</entent>','<recommender>','<topic>',"</topic>",'<seeker>',"no-topic",'<movies>','</movies>']+['<profile>','</profile>',"[Movie]","<big5>","</big5>"]


In [ ]:
#Get the intent unique tokens to add them to the tokenizer
intent_tokens=[]
for i,dial in enumerate(TGD_All_data["train"]):
    goal_path=dial["goal_path"]    
    for k,v in goal_path.items():
      
      intent_tokens=intent_tokens+v["intents"]
intent_tokens=np.unique(intent_tokens)

In [ ]:
#Instanciate the DialoGPT-small pre-trqined model tokenizer
tokenizer = AutoTokenizer.from_pretrained('microsoft/DialoGPT-small')


Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/641 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

In [ ]:
#get all the topic tokens that are in our dataset and that they do not exist in the DialoGPT-small pre-trqined model tokenizer
topics_not_existed=[x  for x in TGD_All_data['real_topics'].values() if x not in tokenizer.get_vocab().keys()]


In [ ]:
# define the non existed topics, the intents tokens, the control special tokens and the movie ids tokens in a one token set so that we add them to the tokenizer vocabulary
vocab_to_add=topics_not_existed+list(intent_tokens)+special_tokens+list(TGD_All_data["all_movies"].keys())


In [ ]:
#Update the tokenizer vocabulary by adding the vocab-to-add tokens
tokenizer.add_tokens(vocab_to_add)



35796

In [ ]:
#Define the '[PAD]' as a padding tokens
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

1

In [ ]:
#Reshape the conversation utterances into a one text by sepparating the issterances with the DialoGPT pre-trqined model End of sentence token
def construct_conv(rows, tokenizer):
    all_rows=[]
    for j,row in enumerate(rows):
      conv = "".join(x+tokenizer.decode(tokenizer.eos_token_id) for x in row).replace(' .','.').replace(' ,',',')
      conv=re.sub(' +', ' ', conv).strip()
      all_rows.append(conv)
    return all_rows
train_data=construct_conv(train_rows,tokenizer) 
test_data=construct_conv(test_rows,tokenizer) 
valid_data=construct_conv(valid_rows,tokenizer) 

In [ ]:
#Shuffel the data randomly
random.shuffle(train_data)
random.shuffle(test_data)
random.shuffle(valid_data)

In [ ]:
#Tokenize the input using the pre-trained DialoGPT model tokenizer to gen the tokenized vector and the attention vector
def tokenize_inputs(liste, tokenizer, num_embeddings=1024):
    
    outputs = tokenizer.batch_encode_plus(liste,truncation=True,return_tensors='pt',return_attention_mask=True,max_length=num_embeddings, padding='max_length')
    input_ids =outputs.input_ids
    outputs["labels"]=input_ids
    

    return (input_ids,outputs.attention_mask,outputs.labels)
train_input_ids,train_attention_masks,train_labels=tokenize_inputs(train_data, tokenizer=tokenizer, num_embeddings=512)
test_input_ids,test_attention_masks,test_labels=tokenize_inputs(test_data, tokenizer=tokenizer, num_embeddings=512)
valid_input_ids,valid_attention_masks,valid_labels=tokenize_inputs(valid_data, tokenizer=tokenizer, num_embeddings=512)

In [ ]:
#Rename the data for shortness purposes
X_train,train_masks,Y_train= (train_input_ids,train_attention_masks,train_labels)
X_test,test_masks,Y_test= (test_input_ids,test_attention_masks,test_labels)
X_valid,valid_masks,Y_valid= (valid_input_ids,valid_attention_masks,valid_labels)

In [ ]:
#Reformulate the data in a batchsize to make it acceptable by the pytorch framework to train our model
batchsize=5

train_data = TensorDataset(X_train, train_masks, Y_train)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data,\
                              sampler=train_sampler,\
                              batch_size=batchsize)

validation_data = TensorDataset(X_valid, valid_masks, Y_valid)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data,\
                                   sampler=validation_sampler,\
                                   batch_size=batchsize)

batchsize=5
testing_data = TensorDataset(X_test, test_masks, Y_test)
test_sampler = SequentialSampler(testing_data)
test_dataloader = DataLoader(testing_data,\
                                   sampler=test_sampler,\
                                   batch_size=batchsize)

# Desing the model training pipeline

In [ ]:
#Design the dialogue generation model
class DialogGPT(torch.nn.Module):
  
  def __init__(self,tokenizer):
    super(DialogGPT, self).__init__()
    
    self.dialogGPT = AutoModelForCausalLM.from_pretrained('microsoft/DialoGPT-small')# Instanciate the pre-trained DialoGPT-small model
    self.dialogGPT.resize_token_embeddings(len(tokenizer))# update the model vocabulary by adding the special tokens that we add to the tokenizer

   

   
  def forward(self, input_ids, token_type_ids=None,\
              attention_mask=None, labels=None): 
   
        
    outputs = self.dialogGPT(input_ids,attention_mask=attention_mask, labels=labels)
    loss = outputs[0]
      
      
    return loss



  


model = DialogGPT(tokenizer=tokenizer) #Instanciate the model
model.to(device)# Indicate that the model will be runing on our specified device

Downloading:   0%|          | 0.00/335M [00:00<?, ?B/s]

DialogGPT(
  (dialogGPT): GPT2LMHeadModel(
    (transformer): GPT2Model(
      (wte): Embedding(86054, 768)
      (wpe): Embedding(1024, 768)
      (drop): Dropout(p=0.1, inplace=False)
      (h): ModuleList(
        (0): GPT2Block(
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (attn): GPT2Attention(
            (c_attn): Conv1D()
            (c_proj): Conv1D()
            (attn_dropout): Dropout(p=0.1, inplace=False)
            (resid_dropout): Dropout(p=0.1, inplace=False)
          )
          (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): GPT2MLP(
            (c_fc): Conv1D()
            (c_proj): Conv1D()
            (act): NewGELUActivation()
            (dropout): Dropout(p=0.1, inplace=False)
          )
        )
        (1): GPT2Block(
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (attn): GPT2Attention(
            (c_attn): Conv1D()
            (c_proj): Conv1D()
        

In [ ]:
optimizer = AdamW(model.parameters(), lr=1e-5, weight_decay=0.01, correct_bias=False) #Define the training optimizer

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [ ]:
#Define the training algorithm
#The training method code is copied from this link : https://towardsdatascience.com/multi-label-text-classification-with-xlnet-b5f5755302df

def train(model, num_epochs,\
          optimizer,\
          train_dataloader, valid_dataloader,\
          model_save_path,\
          train_loss_set=[], valid_loss_set = [],\
          lowest_eval_loss=None, start_epoch=0,\
          device="cpu"
          ):
  """
  Train the model and save the model with the lowest validation loss
  """

  model.to(device)

  for i in trange(num_epochs, desc="Epoch"):
    
    actual_epoch = start_epoch + i

    

    model.train()

    tr_loss = 0
    num_train_samples = 0

    for step, batch in enumerate(train_dataloader):
      # Add batch to GPU
      batch = tuple(t.to(device) for t in batch)
      # Unpack the inputs from our dataloader
      b_input_ids, b_input_mask, b_labels_topics = batch
      # Clear out the gradients (by default they accumulate)
      optimizer.zero_grad()
      # Forward pass
      loss = model(b_input_ids, attention_mask=b_input_mask, labels=b_labels_topics)
      # store train loss
      tr_loss += loss.item()
      num_train_samples += b_labels_topics.size(0)
      # Backward pass
      loss.backward()
      # Update parameters and take a step using the computed gradient
      optimizer.step()
      #scheduler.step()

    # Update tracking variables
    epoch_train_loss = tr_loss/num_train_samples
    train_loss_set.append(epoch_train_loss)

    print("Train loss: {}".format(epoch_train_loss))

    # Validation

    # Put model in evaluation mode to evaluate loss on the validation set
    model.eval()

    # Tracking variables 
    eval_loss = 0
    num_eval_samples = 0

    # Evaluate data for one epoch
    for batch in valid_dataloader:
      # Add batch to GPU
      batch = tuple(t.to(device) for t in batch)
      # Unpack the inputs from our dataloader
      b_input_ids, b_input_mask, b_labels_topics = batch
      # Telling the model not to compute or store gradients,
      # saving memory and speeding up validation
      with torch.no_grad():
        # Forward pass, calculate validation loss
        loss= model(b_input_ids, attention_mask=b_input_mask, labels=b_labels_topics)
        # store valid loss
        eval_loss += loss.item()
        num_eval_samples += b_labels_topics.size(0)

    epoch_eval_loss = eval_loss/num_eval_samples
    valid_loss_set.append(epoch_eval_loss)

    print("Valid loss: {}".format(epoch_eval_loss))

    if lowest_eval_loss == None:
      lowest_eval_loss = epoch_eval_loss
      # save model
      save_model(model, model_save_path, actual_epoch,\
                 lowest_eval_loss, train_loss_set, valid_loss_set)
    else:
      if epoch_eval_loss < lowest_eval_loss:
        lowest_eval_loss = epoch_eval_loss
        # save model
        save_model(model, model_save_path, actual_epoch,\
                   lowest_eval_loss, train_loss_set, valid_loss_set)
    print("\n")

  return model, train_loss_set, valid_loss_set


def save_model(model, save_path, epochs, lowest_eval_loss, train_loss_hist, valid_loss_hist):
  """
  Save the model to the path directory provided
  """
  model_to_save = model.module if hasattr(model, 'module') else model
  checkpoint = {'epochs': epochs, \
                'lowest_eval_loss': lowest_eval_loss,\
                'state_dict': model_to_save.state_dict(),\
                'train_loss_hist': train_loss_hist,\
                'valid_loss_hist': valid_loss_hist
               }
  torch.save(checkpoint, save_path)
  print("Saving model at epoch {} with validation loss of {}".format(epochs,\
                                                                     lowest_eval_loss))
  return
  
def load_model(save_path,tokenizer):
  """
  Load the model from the path directory provided
  """
  checkpoint = torch.load(save_path)
  model_state_dict = checkpoint['state_dict']
  model = DialogGPT(tokenizer)
  model.load_state_dict(model_state_dict)

  epochs = checkpoint["epochs"]
  lowest_eval_loss = checkpoint["lowest_eval_loss"]
  train_loss_hist = checkpoint["train_loss_hist"]
  valid_loss_hist = checkpoint["valid_loss_hist"]
  
  return model, epochs, lowest_eval_loss, train_loss_hist, valid_loss_hist

In [ ]:
#Define the training epochs and the stroing path
import os
num_epochs=50

cwd = os.getcwd()

model_save_path = output_model_file = os.path.join(cwd, "drive/MyDrive/Models/DialogGeneration/Prompt_based_TGD_small_keep_all_data2.bin")


In [ ]:
#Train the model
model, train_loss_set, valid_loss_set = train(model=model,\
                                              num_epochs=num_epochs,\
                                              optimizer=optimizer,\
                                              train_dataloader=train_dataloader,\
                                              valid_dataloader=validation_dataloader,\
                                              model_save_path=model_save_path,\
                                              device=device)
#0.016783231785479146

Epoch:   0%|          | 0/50 [00:00<?, ?it/s]

Train loss: 1.2846898829677373
Valid loss: 0.6625295302669469


Epoch:   2%|▏         | 1/50 [15:39<12:46:52, 939.04s/it]

Saving model at epoch 0 with validation loss of 0.6625295302669469


Train loss: 0.6171981385765951
Valid loss: 0.5180989647163583


Epoch:   4%|▍         | 2/50 [31:17<12:31:02, 938.80s/it]

...

Saving model at epoch 47 with validation loss of 0.2836445113311828


Train loss: 0.2358492140562272


Epoch:  98%|█████████▊| 49/50 [12:46:56<15:38, 938.70s/it]

Valid loss: 0.28400519628033455


Train loss: 0.23439544036572227


Epoch: 100%|██████████| 50/50 [13:02:34<00:00, 939.09s/it]

Valid loss: 0.28419453952869933




# Model evaluation

In [ ]:
# Restore the best model trained version
#This code is borrowed from this link : https://towardsdatascience.com/multi-label-text-classification-with-xlnet-b5f5755302df

checkpoint = torch.load(model_save_path)
model_state_dict = checkpoint['state_dict']
model = DialogGPT(tokenizer=tokenizer)
model.load_state_dict(model_state_dict)
model.to(device)

DialogGPT(
  (dialogGPT): GPT2LMHeadModel(
    (transformer): GPT2Model(
      (wte): Embedding(86054, 768)
      (wpe): Embedding(1024, 768)
      (drop): Dropout(p=0.1, inplace=False)
      (h): ModuleList(
        (0): GPT2Block(
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (attn): GPT2Attention(
            (c_attn): Conv1D()
            (c_proj): Conv1D()
            (attn_dropout): Dropout(p=0.1, inplace=False)
            (resid_dropout): Dropout(p=0.1, inplace=False)
          )
          (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): GPT2MLP(
            (c_fc): Conv1D()
            (c_proj): Conv1D()
            (act): NewGELUActivation()
            (dropout): Dropout(p=0.1, inplace=False)
          )
        )
        (1): GPT2Block(
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (attn): GPT2Attention(
            (c_attn): Conv1D()
            (c_proj): Conv1D()
        

In [ ]:
checkpoint['train_loss_hist']

[1.2846898829677373,
 0.6171981385765951,
 0.49874861839590806,
 0.4320730067618529,
 0.3967288671485672,
 0.3740562236751087,
 0.3582311083668467,
 0.346072169736668,
 0.33668203339848957,
 0.32902071718751436,
 0.3227101239967234,
 0.31733727956112867,
 0.31257122424576406,
 0.3083085085519136,
 0.3045182793472991,
 0.3010301900232729,
 0.29774168651899635,
 0.29470464296380233,
 0.2918316796641268,
 0.28907199216071405,
 0.2865110694555201,
 0.2840430421082675,
 0.2816550724081182,
 0.2793685955886212,
 0.27718543039482435,
 0.2750143622425881,
 0.27292964939793257,
 0.27091925109113363,
 0.26894562447892,
 0.2669540385487081,
 0.2651936978294682,
 0.26342746365273817,
 0.261542345777268,
 0.25975854870429665,
 0.2581057334381247,
 0.25630509835120296,
 0.2546360231975445,
 0.25295213752806084,
 0.251345119025863,
 0.24966870358300391,
 0.24809039858525608,
 0.2464956998263758,
 0.244940212286522,
 0.24341794450960838,
 0.24190935598112964,
 0.2403670887164048,
 0.23887822499620417,

In [ ]:
def perplexity(pre_trqined_model,valid_dataloader):
    model=pre_trqined_model
    model.eval()
    eval_loss = 0
    num_eval_samples = 0
    nb_eval_steps=0
    for batch in valid_dataloader:
      
      batch = tuple(t.to(device) for t in batch)
      
      b_input_ids, b_input_mask, b_labels_topics = batch
      


      with torch.no_grad():
            outputs = model(b_input_ids, attention_mask=b_input_mask, labels=b_labels_topics)
            lm_loss = outputs
            eval_loss += lm_loss.mean().item()
      nb_eval_steps += 1

    eval_loss = eval_loss / nb_eval_steps
    perplexity = torch.exp(torch.tensor(eval_loss))


      
    
    return perplexity

In [ ]:
perplexity(model,test_dataloader)


tensor(4.1744)

In [ ]:
blue = load_metric('bleu')

In [ ]:
def get_responses(pretrained,rows):
  all_rows=[]
  model=pretrained
  all_responses=[]
  
  for k,dialogue in enumerate(rows):
    print(k)
    current_data=""
    personality=dialogue[0]
    seeker=True
    for i in range(1,len(dialogue),1):
      
      row=dialogue[:i]
      all_rows.append(dialogue[i])
      '''print(f"fed input: {row}")
      print(f"real row: {dialogue[i]}")
      print(f"role real row: {dialogue[i].split()[0]}")'''
      print(f"real row: {dialogue[i]}")
      current_data=" ".join(x for x in row)
      
      input_ids=tokenizer.encode(current_data + tokenizer.eos_token, return_tensors='pt').to('cuda')
      chat_history_ids = model.generate(
        input_ids, max_length=1000,
        pad_token_id=tokenizer.eos_token_id,
        no_repeat_ngram_size=5,
        num_beams=5,
        
        
    )
      response=tokenizer.decode(chat_history_ids[:, input_ids.shape[-1]:][0], skip_special_tokens=True)
      all_responses.append(response)
      print(f"response : {response}")

  return all_responses,all_rows


In [ ]:
predicted_responses,real_responses=get_responses(model.dialogGPT,test_rows)

Streaming output truncated to the last 5000 lines.
response : <recommender> okay, i recommend you to watch @1293414. in this movie, maggie cheung's acting skills are really nothing to say. <topic> recommended movies no-topic </topic>
real row: <seeker> i have seen this movie , and the acting skills of her and tony leung in the film are really at the level of the actor and actress . recommend me an acting movie . <topic> feedback request recommendation acting school </topic>
response : <seeker> i have watched this movie. it is a good movie, but i don’t like it. it’s a pity that the script is not well written. <topic> feedback request recommendation acting school </topic>
real row: <recommender> this small @1301459 , conscience recommends you to see it , the ending of the film is too powerful , and it is decisively praised . <topic> recommended movies no-topic </topic>
response : <recommender> i recommend you to watch the @4023638, the story is very smooth, the acting skills are superb, 

In [ ]:
predicted_responses_splited=[x.split() for x in predicted_responses]
real_responses_splited=[[x.split() ]for x in real_responses]

In [ ]:
metric = load_metric('bleu')

In [ ]:
final_score = metric.compute(predictions=predicted_responses_splited, references=real_responses_splited)

In [ ]:
#
print('Individual 1-gram: %f' % corpus_bleu(real_responses_splited, predicted_responses_splited, weights=(1, 0, 0, 0)))
print('Individual 2-gram: %f' % corpus_bleu(real_responses_splited, predicted_responses_splited, weights=(0, 1, 0, 0)))
print('Individual 3-gram: %f' % corpus_bleu(real_responses_splited, predicted_responses_splited, weights=(0, 0, 1, 0)))


Individual 1-gram: 0.360393
Individual 2-gram: 0.151492
Individual 3-gram: 0.080341


In [ ]:
metric = load_metric("bertscore")
final_score = metric.compute(predictions=predicted_responses,references=real_responses,lang="en")#rescale_with_baseline=True

Downloading:   0%|          | 0.00/482 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

In [ ]:
mean(final_score["recall"])
#

0.9041067383354302

In [ ]:
mean(final_score["precision"])
#

0.9162255897663716

In [ ]:
mean(final_score["f1"])
#

0.9099851033888848

#Test without special tokens

In [ ]:
def delet_special_tokens(predicted_responses,special_tokens):
  new=[]
  for row in predicted_responses:
    if("<profile>" in row) or ("<big5>" in row):
      continue
    else:
    
      for s in special_tokens:
          row=row.replace(s,'')
      row=re.sub(' +', ' ', row).strip()
      new.append(row)
        #print(row.replace(s,''))
  return new

predicted_responses_delet_special_tokns= delet_special_tokens(predicted_responses,special_tokens)
real_responses_delet_special_tokns= delet_special_tokens(real_responses,special_tokens)

In [ ]:
predicted_responses_delet_special_tokns_splited=[x.split() for x in predicted_responses_delet_special_tokns]
real_responses_delet_special_tokns_splited=[[x.split() ]for x in real_responses_delet_special_tokns]

In [ ]:
metric = load_metric('bleu')

In [ ]:
final_score = metric.compute(predictions=predicted_responses_delet_special_tokns_splited, references=real_responses_delet_special_tokns_splited)
print('Individual 1-gram: %f' % corpus_bleu(real_responses_delet_special_tokns_splited, predicted_responses_delet_special_tokns_splited, weights=(1, 0, 0, 0)))
print('Individual 2-gram: %f' % corpus_bleu(real_responses_delet_special_tokns_splited, predicted_responses_delet_special_tokns_splited, weights=(0, 1, 0, 0)))
print('Individual 3-gram: %f' % corpus_bleu(real_responses_delet_special_tokns_splited, predicted_responses_delet_special_tokns_splited, weights=(0, 0, 1, 0)))

Individual 1-gram: 0.267321
Individual 2-gram: 0.093259
Individual 3-gram: 0.036407


In [ ]:
metric = load_metric("bertscore")

In [ ]:
final_score = metric.compute(predictions=predicted_responses_delet_special_tokns,references=real_responses_delet_special_tokns,lang="en")

In [ ]:
mean(final_score["recall"])
#

0.871678435226469

In [ ]:
mean(final_score["precision"])
#

0.8863892528027064

In [ ]:
mean(final_score["f1"])
#

0.8788579994102556

#Test movie prediction

In [ ]:
def movie_completion_performance(pretrained,rows,num):
  all_rows=[]
  model=pretrained
  realmovies=[]
  predictedmovies=[]
  al_hit=0
  al_mrr=0
  
  for k,dialogue in enumerate(rows):
    #print(k)
    mrr=0
    hit=0
    #dialogue=dialogue.replace("<|endoftext|>"," ")
    
    find_all_mentioned_movies= re.findall(r'@[0-9]+', dialogue)
    for k in find_all_mentioned_movies:
      splited=dialogue.split(k)
      fed_input =splited[0]
      input_ids=tokenizer.encode(fed_input , return_tensors='pt').to('cuda')
     # print(f'fed inputs : {fed_input}')
      num_return_sequences=num
      chat_history_ids = model.generate(
          input_ids, max_length=len(input_ids[0])+1,
          pad_token_id=tokenizer.eos_token_id,
          no_repeat_ngram_size=5,
          num_beams=num_return_sequences,
          num_return_sequences=num_return_sequences
          
        
        
    )
      response=[tokenizer.decode(chat_history_ids[:, input_ids.shape[-1]:][l], skip_special_tokens=True) for l in range(num_return_sequences)]

      for w,r in enumerate(response):

        if k in r:
          mrr=mrr+(1/(w+1))
          hit=hit+1
          #print(f'response: {response}')
          predictedmovies.append(response)
          break
        else:
          mrr=mrr
          hit=hit
      mrr=mrr/len(find_all_mentioned_movies)
      hit=hit/len(find_all_mentioned_movies)
    al_mrr=al_mrr+mrr
    al_hit=al_hit+hit

  #print(f"result MRR {(al_mrr/len(rows))}")  
  #print(f"result hit {(al_hit/len(rows))}")  
      

  return ((al_mrr/len(rows)),(al_hit/len(rows)))

In [ ]:
predicte_movies=movie_completion_performance(model.dialogGPT,test_data,num=1)
predicte_movies
#(0.02455216224747474, 0.02455216224747474)


(0.024886386846405227, 0.024886386846405227)

In [ ]:
predicte_movies=movie_completion_performance(model.dialogGPT,test_data,num=3)
predicte_movies
#(0.03296865066102198, 0.04520997567587643)



(0.03316052033818577, 0.04520997567587641)

In [ ]:
predicte_movies=movie_completion_performance(model.dialogGPT,test_data,num=5)
predicte_movies
#(0.03732276557107209, 0.06353750681538367)

(0.03764213574338344, 0.0640574117470533)

In [ ]:
predicte_movies=movie_completion_performance(model.dialogGPT,test_data,num=10)
predicte_movies
#(0.04080625203173381, 0.09018166747838623)

(0.041057613098045856, 0.09025593961148191)

# The Impact of each personality trait on the movie recommendation performance

In [ ]:
def movie_completion_performance_effect_of_big5_traits(pretrained,rows,num,index):
  all_rows=[]
  model=pretrained
  realmovies=[]
  predictedmovies=[]
  al_hit=0
  al_mrr=0
  
  for k,dialogue in enumerate(rows):
    #print(k)
    mrr=0
    hit=0
    #dialogue=dialogue.replace("<|endoftext|>"," ")
    
    find_all_mentioned_movies= re.findall(r'@[0-9]+', dialogue)
    for k in find_all_mentioned_movies:
      splited=dialogue.split(k)
      #print(splited[0])
      s=splited[0].split('</big5>')
      big5=s[0].replace('<big5>','')
      big5=re.sub(' +', ' ', big5).strip()

      big5=big5.split()
      big5[index]=tokenizer.unk_token
      big5= ' '.join(x for x in big5)
      big5=re.sub(' +', ' ', big5).strip()

      fed_input ='<big5> '+big5+' </big5>'+s[1]

      input_ids=tokenizer.encode(fed_input , return_tensors='pt').to('cuda')
      #print(f'fed inputs : {fed_input}')
      num_return_sequences=num
      chat_history_ids = model.generate(
          input_ids, max_length=len(input_ids[0])+1,
          pad_token_id=tokenizer.eos_token_id,
          no_repeat_ngram_size=5,
          num_beams=num_return_sequences,
          num_return_sequences=num_return_sequences
          
        
        
    )
      response=[tokenizer.decode(chat_history_ids[:, input_ids.shape[-1]:][l], skip_special_tokens=True) for l in range(num_return_sequences)]

      for w,r in enumerate(response):

        if k in r:
          mrr=mrr+(1/(w+1))
          hit=hit+1
          #print(f'response: {response}')
          predictedmovies.append(response)
          break
        else:
          mrr=mrr
          hit=hit
      mrr=mrr/len(find_all_mentioned_movies)
      hit=hit/len(find_all_mentioned_movies)
    al_mrr=al_mrr+mrr
    al_hit=al_hit+hit

  #print(f"result MRR {(al_mrr/len(rows))}")  
  #print(f"result hit {(al_hit/len(rows))}")  
      

  return ((al_mrr/len(rows)),(al_hit/len(rows)))

In [ ]:
pers=['AGR','OPN','CSN','EXT','NEU']

In [ ]:
def get_impact_of_personality_trait_on_the_Recommendation(model,test_data,pers):
  
  trait_effect_on_MRR_and_hit={}
  for i,v in enumerate(pers):
    #print(f'trait {v}')
    Mrr_Tab=[]
    Hr_Tab=[]
    for j,k in enumerate([1,5,10]):
      MRR,HR=movie_completion_performance_effect_of_big5_traits(model,test_data,num=k,index=i)
      Mrr_Tab.append(MRR)
      Hr_Tab.append(HR)


    trait_effect_on_MRR_and_hit[v]={'MRR@k':Mrr_Tab,'HR@k':Hr_Tab}
    #print(trait_effect_on_MRR_and_hit[v])

  return trait_effect_on_MRR_and_hit



    
trait_effect_on_MRR_and_hit=get_impact_of_personality_trait_on_the_Recommendation(model.dialogGPT,test_data,pers)
trait_effect_on_MRR_and_hit

{'AGR': {'HR@k': [0.0254434278446227,
   0.062144556781045766,
   0.09109106878310005],
  'MRR@k': [0.0254434278446227, 0.037972913881461665, 0.04177956669844385]},
 'CSN': {'HR@k': [0.024774978646761737,
   0.060579534939764736,
   0.09376486557454392],
  'MRR@k': [0.024774978646761737, 0.03764608664399043, 0.04198672915099325]},
 'EXT': {'HR@k': [0.024267452403941372,
   0.0633647399308521,
   0.09515127872566295],
  'MRR@k': [0.024267452403941372, 0.03764154779141237, 0.04164697221146098]},
 'NEU': {'HR@k': [0.02470070651366607,
   0.0625477464667998,
   0.09314593113208007],
  'MRR@k': [0.02470070651366607, 0.038002592882849615, 0.042017734588995465]},
 'OPN': {'HR@k': [0.025270126200732816,
   0.062312551378663525,
   0.09297978591768116],
  'MRR@k': [0.025270126200732816, 0.03802858812943309, 0.04215019692130843]}}